In [174]:
import gensim
import os
import collections
import smart_open
import random

import pandas as pd
import numpy as np


from pprint import pprint
import string
translator = str.maketrans('', '', string.punctuation)

# TODO
- Remove punctuations
- Filtering based on word count

In [175]:
def get_train_data():
    """
    SQL here 
    """
    data = pd.read_csv('~/Desktop/grocery_products.csv')
    return data

In [176]:
data = get_train_data()

In [177]:
print(data.shape)

(893626, 17)


In [213]:
selected_data = data[['source_product_id', 'name', 'departments_name', 'categories_name', 'description_en']]
selected_data = selected_data.head(1000)

selected_data

,source_product_id,name,departments_name,categories_name,description_en
0,2506104,A08. Golden Nasi Super Combo,Department - (Mchefz Cuisine Menu (mchefz-cuis...,Rice,Served with fried chicken thigh and braised be...
1,2682483,Honey Stars Breakfast Cereal,Food Cupboard,Cereals & Cereal Bars,This product is getting close to its expiry da...
2,2683014,Ice Lemon Tea,Drinks,Tea,Refreshing combination of fruity flavours and ...
3,2768893,Anticavity Maximum Cavity Protection Toothpaste,Health & Beauty,Oral Care,This product is getting close to its expiry da...
4,2771143,Daily Moisture Therapy Dermo-Cleanser Twin Pack,Health & Beauty,Body Care,Daily Moisture Therapy Dermo-Cleanser is a hyp...
5,2875875,Bak Kut Teh Spices Traditional,Food Cupboard,Sauces & Dressings,Enjoy a bowl of Bak Kut Teh at any time of the...
6,2877983,Paste - Fish Curry,Members Specials,Food Cupboard,"Prepare aromatic, spicy, mouth-watering fish c..."
7,2536227,S8 Yaw Mug Lettuce,Department - (Delivery Menu (kee-hiong-klang-b...,Side Dish,NaN
8,2682411,Mackerel in Tomato Sauce,Food Cupboard,Canned & Packaged Food,NaN
9,3013595,Tender Gammon Ham Boneless,"Dairy, Eggs & Chilled Food",Chilled Meat & Seafood,NaN


Creating document string

In [214]:
selected_data['document'] = selected_data['name'] + ' ' + selected_data['departments_name'] + ' ' + selected_data['categories_name'] + ' ' + selected_data['description_en']

In [215]:
docs = selected_data['document'].tolist()

In [197]:
selected_data[['document']].iloc[3]

document    Anticavity Maximum Cavity Protection Toothpast...
Name: 3, dtype: object

In [216]:
pp.pprint(docs)
# print(type(docs[0]))

[ 'A08. Golden Nasi Super Combo Department - (Mchefz Cuisine Menu '
  '(mchefz-cuisine)) Rice Served with fried chicken thigh and braised beef and '
  'cuttlefish, add ons available',
  'Honey Stars Breakfast Cereal Food Cupboard Cereals & Cereal Bars This '
  "product is getting close to its expiry date—don't let this get away from "
  "you! It's still safe for consumption, now enjoy it at a lower "
  'price.<br/><br/>Expiry Date: 2018-07-30<br/><br/>',
  'Ice Lemon Tea Drinks Tea Refreshing combination of fruity flavours and real '
  'tea leaves. Sweet cooling tea taste with a stimulating lemon flavour finish',
  'Anticavity Maximum Cavity Protection Toothpaste Health & Beauty Oral Care '
  "This product is getting close to its expiry date—don't let this get away "
  "from you! It's still safe for use, now enjoy it at a lower price.<br/> "
  '<br/>Expiry Date : 2018-10-13',
  'Daily Moisture Therapy Dermo-Cleanser Twin Pack Health & Beauty Body Care '
  'Daily Moisture Therapy Dermo-

  'and stews. Chopped coriander leaves is also commonly used as a garnish for '
  'Indian dishes such as Dal. Rich in antioxidants, essential oils and '
  'vitamins.',
  'Sweet Potatoes, Broccoli & Carrots Pouch 4+ Months Babies Food & '
  "Supplements Make your baby's first meal healthy and enjoyable with Ella's "
  'Kitchen Organic Sweet Potatoes, Broccoli & Carrots. This delicious and '
  'nutritious baby food is made of pureed sweet potatoes, broccoli and '
  "carrots. It's 100% pure and organic. No dairy, egg, wheat, lactose or "
  'gluten. Comes in a convenient pouch.',
  'Irish Oatmeal - Steel Cut Food Cupboard Cereals & Cereal Bars Ingredients: '
  '100% Whole grain Irish Oats',
  'Chocolate Coated Muesli Bars - Chocolate & Nut Food Cupboard Cereals & '
  'Cereal Bars Yogood Choc&Nut features the delightful blend of cereals and '
  'mixed nuts coated with creamy milk chocolate. We select only the finest '
  'ingredients for Yogood Choc&Nut to bring you a nutty, chewy and delici

  'for purity and taste. With 24 in a pack, these 600ml bottle are the perfect '
  'size to take out and about.',
  'BBQ Pork Pau (6 pcs) Dairy, Eggs & Chilled Food Chilled Noodles, Pastry, '
  'Buns Country of Orgin: Singapore',
  'Apricot & Almond Cream Cheese Dairy, Eggs & Chilled Food Cheese Country of '
  'Orgin: Australia',
  nan,
  'Organic Gluten - Free Corn Quinoa Elbows Food Cupboard Rice, Noodles, Pasta '
  '& Rolls Great for making your own mac and cheese or something a little more '
  'elevated for date night, this corn-quinoa pasta has all the taste and '
  'texture you want with none of the gluten you don’t.',
  'Magnolia - Vanilla Flavoured Ice Confection Members Specials Frozen Food '
  'This super-sized 1.5L tub of Magnolia Vanilla Ice Cream is for sharing, '
  'perfect for your next family reunion and guaranteed to please all. This is '
  'a classically smooth and creamy Magnolia Vanilla ice cream, versatile and '
  'delicious this ice cream is perfect anyway you wan

In [217]:
docs = [d.translate(translator) for d in docs if isinstance(d, str)]

In [218]:
pprint(docs)

['A08 Golden Nasi Super Combo Department  Mchefz Cuisine Menu mchefzcuisine '
 'Rice Served with fried chicken thigh and braised beef and cuttlefish add ons '
 'available',
 'Honey Stars Breakfast Cereal Food Cupboard Cereals  Cereal Bars This product '
 'is getting close to its expiry date—dont let this get away from you Its '
 'still safe for consumption now enjoy it at a lower pricebrbrExpiry Date '
 '20180730brbr',
 'Ice Lemon Tea Drinks Tea Refreshing combination of fruity flavours and real '
 'tea leaves Sweet cooling tea taste with a stimulating lemon flavour finish',
 'Anticavity Maximum Cavity Protection Toothpaste Health  Beauty Oral Care '
 'This product is getting close to its expiry date—dont let this get away from '
 'you Its still safe for use now enjoy it at a lower pricebr brExpiry Date  '
 '20181013',
 'Daily Moisture Therapy DermoCleanser Twin Pack Health  Beauty Body Care '
 'Daily Moisture Therapy DermoCleanser is a hypoallergenic soapfree cleanser '
 'ideal for dr

 'Olive Oil  White Truffle Food Cupboard Sauces  Dressings Ingredients Olive '
 'oil aroma',
 'Plantain Chips Snacks  Sweets Dried Snacks This tasty snack is made of '
 'plantain slices a bananas cousin delicately roasted in pure vegetable oil '
 'and sea salt Enjoy our light crispy crunchy snacks with a nutty flavour for '
 'the zest conscious Be part of our world… reward yourself',
 'Processed Cheese 8 Portion Members Specials Dairy Eggs  Chilled Food Spread '
 'on toast or eat whole as a snack Cowhead processed cheese portions add a '
 'creamy cheese to your kitchen Soft thick cheese portions are an enjoyable '
 'way to eat more protein and calcium Cowhead is a homegrown Asia Pacific '
 'dairy brand Produced under strict quality requirements cowhead processed '
 'cheese portionscome in packs of eight individuallywrapped triangles Theyre a '
 'tasty  snacking option for cheeselovers',
 'Green Capsicum Fresh Vegetables Capsicum  Chillies Bright bold and beautiful '
 'the Green Capsicu

 'Oats wheat flour sugar vegetable oil Hydrogenated partially hydrogenated '
 'corn',
 'Premium Organic Sprouted Tau Kwa Members Specials Dairy Eggs  Chilled Food '
 'Vitasoy tau kwa tofu is made with nonGMO sprouted tofu Quickly and easily '
 'prepare delicious tofu by pan frying or deep frying This tau kwa tofu '
 'naturally contains no transfats or preservatives It is 25 lower in saturated '
 'fats than other tofu products making this a smart healthy option for a '
 'speedy meal Easypeel packaging makes it even simpler to enjoy sprouted tau '
 'kwa tofu',
 'Organic Creamed Coconut Food Cupboard Cooking Ingredients Rich and creamy '
 'organic coconut milk and creamed coconut from Sri Lanka Tasty and wit a '
 'natural emulgator Coconut milk is an essentiel ingredient for all kinds of '
 'oriental dishes',
 'Organic Virgin Coconut Oil Members Specials Food Cupboard Dietary coconut '
 'oil for health made from the best strain coconut from tapee basin produced '
 'by standardized cold pr

In [219]:
docs[3]

'Anticavity Maximum Cavity Protection Toothpaste Health  Beauty Oral Care This product is getting close to its expiry date—dont let this get away from you Its still safe for use now enjoy it at a lower pricebr brExpiry Date  20181013'

In [220]:
def clean_documents(docs):
    # remove common words and tokenize
    stoplist = set('for a of the and to in'.split())
    docs = [[word for word in document.lower().split() if word not in stoplist]
             for document in docs]

    # remove words that appear only once
    from collections import defaultdict
    frequency = defaultdict(int)
    for text in docs:
        for token in text:
            frequency[token] += 2

    docs = [[token for token in text if frequency[token] > 1] for text in docs]

    pprint(docs)
    return docs

In [221]:
docs = clean_documents(docs)

[['a08',
  'golden',
  'nasi',
  'super',
  'combo',
  'department',
  'mchefz',
  'cuisine',
  'menu',
  'mchefzcuisine',
  'rice',
  'served',
  'with',
  'fried',
  'chicken',
  'thigh',
  'braised',
  'beef',
  'cuttlefish',
  'add',
  'ons',
  'available'],
 ['honey',
  'stars',
  'breakfast',
  'cereal',
  'food',
  'cupboard',
  'cereals',
  'cereal',
  'bars',
  'this',
  'product',
  'is',
  'getting',
  'close',
  'its',
  'expiry',
  'date—dont',
  'let',
  'this',
  'get',
  'away',
  'from',
  'you',
  'its',
  'still',
  'safe',
  'consumption',
  'now',
  'enjoy',
  'it',
  'at',
  'lower',
  'pricebrbrexpiry',
  'date',
  '20180730brbr'],
 ['ice',
  'lemon',
  'tea',
  'drinks',
  'tea',
  'refreshing',
  'combination',
  'fruity',
  'flavours',
  'real',
  'tea',
  'leaves',
  'sweet',
  'cooling',
  'tea',
  'taste',
  'with',
  'stimulating',
  'lemon',
  'flavour',
  'finish'],
 ['anticavity',
  'maximum',
  'cavity',
  'protection',
  'toothpaste',
  'health',
  'b

  'cook',
  'before',
  'consumption'],
 ['cookie',
  'coconut',
  'snacks',
  'sweets',
  'cookies',
  'biscuits',
  'nongmo',
  'ingredients',
  'dairyfree',
  'vegan',
  'no',
  'preservatives'],
 ['instant',
  'malt',
  'drink',
  'powder',
  'chocolate',
  'members',
  'specials',
  'drinks',
  'over',
  'one',
  'hundred',
  'years',
  'ovaltine',
  'has',
  'been',
  'tried',
  'trusted',
  'hot',
  'drink',
  'choice',
  'many',
  'families',
  'all',
  'over',
  'world',
  'soothing',
  'blend',
  'malt',
  'cocoa',
  'it',
  'has',
  'earned',
  'itself',
  'tagline',
  '‘huginamug’',
  'simply',
  'add',
  'hot',
  'milk',
  'you',
  'will',
  'have',
  'warm',
  'comforting',
  'drink',
  'packed',
  'full',
  'with',
  'healthy',
  'complex',
  'carbohydrates',
  'calcium',
  'naturally',
  'occurring',
  'vitamins',
  'this',
  'nutritious',
  'blend',
  'makes',
  'ovaltine',
  'perfect',
  'choice',
  'your',
  'little',
  'big',
  'ones'],
 ['bodywash',
  'refill',
  '

  'food',
  'cupboard',
  'cereals',
  'cereal',
  'bars',
  'yummy',
  'organic',
  'whole',
  'grain',
  'squares',
  'lightly',
  'sprinkled',
  'with',
  'cinnamon',
  'sugar'],
 ['japanesestyle',
  'udon',
  'soup',
  'bowl',
  'with',
  'cooked',
  'udon',
  'noodles',
  'food',
  'cupboard',
  'rice',
  'noodles',
  'pasta',
  'rolls',
  'comforting',
  'medley',
  'tofu',
  'vegetables',
  'delicate',
  'broth'],
 ['ultra',
  'aromatherapy',
  'romantic',
  'concentrated',
  'fabric',
  'conditioner',
  'household',
  'cleaning',
  'laundry',
  'add',
  'comfort',
  'fabric',
  'conditioner',
  'your',
  'laundry',
  'soften',
  'clothes',
  'reduce',
  'static',
  'infuse',
  'beautiful',
  'scent',
  'comfort',
  'makes',
  'your',
  'clothes',
  'smell',
  'fresh',
  'delicious',
  'all',
  'day',
  'long',
  'this',
  'aromatherapy',
  'romantic',
  'scent',
  'is',
  'blended',
  'by',
  'scent',
  'experts',
  'with',
  'real',
  'essential',
  'oils',
  'romantic',
  'fl

  'cupboard',
  'cooking',
  'ingredients',
  'fennel',
  'seeds',
  'are',
  'an',
  'extremely',
  'useful',
  'addition',
  'any',
  'seasoning',
  'collection',
  'not',
  'only',
  'are',
  'fennel',
  'seeds',
  'an',
  'excellent',
  'dry',
  'rub',
  'meat',
  'but',
  'also',
  'due',
  'their',
  'liquoricelike',
  'flavour',
  'they',
  'can',
  'also',
  'be',
  'used',
  'make',
  'tea',
  'sprinkle',
  'them',
  'on',
  'cakes',
  'add',
  'them',
  'salads',
  'scatter',
  'them',
  'over',
  'vegetables',
  'fennel',
  'seeds',
  'can',
  'also',
  'be',
  'roasted',
  'enjoyed',
  'after',
  'meal',
  'ensuring',
  'fresh',
  'breath',
  'improved',
  'digestion'],
 ['fruit',
  'spread',
  'seedless',
  'red',
  'raspberry',
  'food',
  'cupboard',
  'honey',
  'jam',
  'spreads',
  'fruit',
  'comes',
  'first',
  'when',
  'we',
  'decided',
  'bring',
  'our',
  'decades',
  'organic',
  'fruit',
  'expertise',
  'fruit',
  'spreads',
  'we',
  'felt',
  'strongly',

  'fluid',
  'so',
  'they',
  'light',
  'fast',
  'stay',
  'lit',
  'j'],
 ['minis',
  'creamkakao',
  '20',
  'pieces',
  'snacks',
  'sweets',
  'cookies',
  'biscuits',
  'there',
  'is',
  'reason',
  'why',
  'theobromas',
  'or',
  'food',
  'gods',
  'is',
  'scientific',
  'name',
  'cocoa',
  'tree',
  'loacker',
  'minis',
  'cremkakao',
  'wafer',
  'biscuits',
  'are',
  'made',
  'from',
  'four',
  'layers',
  'crispy',
  'wafers',
  'incasing',
  'three',
  'layers',
  'smooth',
  'creamy',
  'cocoa',
  'chocolate',
  'blend',
  'filling',
  'these',
  'irresistible',
  'miniwafer',
  'biscuits',
  'promise',
  'divine',
  'pleasure',
  'really',
  'are',
  'food',
  'gods'],
 ['beanz',
  'organic',
  'food',
  'cupboard',
  'canned',
  'packaged',
  'food',
  'ingredients',
  'beans',
  '52',
  'tomatoes',
  '33',
  'water',
  'sugar',
  'conrflour',
  'salt',
  'spirit',
  'vinegar'],
 ['caramel',
  'flavour',
  'food',
  'cupboard',
  'baking',
  'ingredients',
  '

...

  'fusilli',
  'food',
  'cupboard',
  'rice',
  'noodles',
  'pasta',
  'rolls',
  'with',
  'unique',
  'shape',
  'delicious',
  'fuller',
  'taste',
  'naturel',
  'fusilli',
  'pasta',
  'is',
  'your',
  'choice',
  'special',
  'lunch',
  'or',
  'dinner',
  'its',
  'made',
  'with',
  'premium',
  'italian',
  'durum',
  'wheat',
  'thats',
  'sodium',
  'free',
  'low',
  'fat',
  'great',
  'source',
  'dietary',
  'fibre',
  'quick',
  'simple',
  'prepare',
  'serve',
  'it',
  'with',
  'your',
  'favourite',
  'pasta',
  'sauce'],
 ['preserved',
  'plum',
  'seedless',
  'food',
  'cupboard',
  'preserved',
  'dried',
  'goods',
  'nyan',
  'preserved',
  'plums',
  'are',
  'individually',
  'wrapped',
  'preserved',
  'plum',
  'sweets',
  'with',
  'slight',
  'clove',
  'taste',
  'these',
  'preserved',
  'plums',
  'are',
  'prepared',
  'without',
  'their',
  'seeds',
  'easy',
  'enjoyment',
  'making',
  'them',
  'good',
  'option',
  'sweet',
  'bowls',
  'or

  'rice',
  'noodles',
  'pasta',
  'rolls',
  'created',
  'over',
  '70',
  'years',
  'ago',
  'uncle',
  'ben’s',
  'was',
  'named',
  'after',
  'texan',
  'rice',
  'farmer',
  'who',
  'decades',
  'earlier',
  'had',
  'gained',
  'reputation',
  'producing',
  'highquality',
  'rice',
  'company',
  'is',
  'currently',
  'one',
  'worlds',
  'largest',
  'rice',
  'brands',
  'uncle',
  'bens',
  'rice',
  'time',
  'mexican',
  'chilli',
  'con',
  'carne',
  'is',
  'microwaveable',
  'two',
  'pot',
  'meal',
  'it',
  'consists',
  'chilli',
  'con',
  'carne',
  'with',
  'rice',
  'separate',
  'pot',
  'simply',
  'peel',
  'back',
  'lid',
  'place',
  'both',
  'pots',
  'microwave',
  'pour',
  'sauce',
  'over',
  'rice',
  '90',
  'seconds',
  'later',
  'enjoy',
  'its',
  'perfect',
  'choice',
  'anyone',
  'with',
  'busy',
  'lifestyle',
  'super',
  'tasty',
  'super',
  'convenient'],
 ['orange',
  'juice',
  'some',
  'pulp',
  'dairy',
  'eggs',
  'chill

  'breads'],
 ['natural',
  'mineral',
  'water',
  'drinks',
  'water',
  'sparkling',
  'water',
  'spritzer',
  'is',
  'pure',
  'natural',
  'mineral',
  'water',
  'great',
  'way',
  'naturally',
  'rehydrate',
  'spritzer',
  'water',
  'is',
  'bestselling',
  'natural',
  'mineral',
  'water',
  'malasia',
  'extracted',
  'over',
  '400',
  'foot',
  'underground',
  'spritzer',
  'is',
  'rich',
  'minerals',
  'hygienically',
  'filtered',
  'purity',
  'taste',
  'with',
  '24',
  'pack',
  'these',
  '600ml',
  'bottle',
  'are',
  'perfect',
  'size',
  'take',
  'out',
  'about'],
 ['bbq',
  'pork',
  'pau',
  '6',
  'pcs',
  'dairy',
  'eggs',
  'chilled',
  'food',
  'chilled',
  'noodles',
  'pastry',
  'buns',
  'country',
  'orgin',
  'singapore'],
 ['apricot',
  'almond',
  'cream',
  'cheese',
  'dairy',
  'eggs',
  'chilled',
  'food',
  'cheese',
  'country',
  'orgin',
  'australia'],
 ['organic',
  'gluten',
  'free',
  'corn',
  'quinoa',
  'elbows',
  'foo

  'goto',
  'brand',
  'fish',
  'fingers',
  'all',
  'over',
  'world',
  'their',
  'crispy',
  'rectangular',
  'fish',
  'patties',
  'coated',
  'breadcrumbs',
  'ready',
  'bake',
  'are',
  'dinnertime',
  'favourite',
  'kids',
  'everywhere',
  'perfect',
  'afternoon',
  'snack',
  'this',
  '10pack',
  'haddock',
  'fish',
  'fingers',
  'is',
  'an',
  'essential',
  'domestic',
  'freezer',
  'covering',
  'you',
  'any',
  'meal',
  'emergency',
  'delicious',
  'dipped',
  'ketchup',
  'or',
  'with',
  'sharper',
  'tartar',
  'sauce'],
 ['pororo',
  'friends',
  'biscuit',
  'plain',
  'flavour',
  'snacks',
  'sweets',
  'cookies',
  'biscuits',
  'ingredients',
  'wheat',
  'flour',
  'buttery',
  'spread',
  'mixed',
  'oil',
  'shortening',
  'whole',
  'milk',
  'powder',
  'dried',
  'cream',
  'extract',
  'ammonium',
  'bicarbonate',
  'egg',
  'shell',
  'calcium',
  'sodium',
  'bicarbonate',
  'soy',
  'lecithin',
  'colostrum',
  'powder',
  'vitamin',
  '

  'time',
  'day'],
 ['i',
  'chefferini',
  'rigati',
  'n38',
  'pasta',
  'di',
  'semola',
  'di',
  'grano',
  'duro',
  'food',
  'cupboard',
  'rice',
  'noodles',
  'pasta',
  'rolls',
  'ingredients',
  'durum',
  'wheat',
  'semolina'],
 ['skin',
  'care',
  'cream',
  'foot',
  'nail',
  'health',
  'beauty',
  'body',
  'care',
  'scholl',
  'cracked',
  'foot',
  'nail',
  'cream',
  'is',
  'specialised',
  'moisturiser',
  'designed',
  'hydrate',
  'soften',
  'dry',
  'skin',
  'maintain',
  'health',
  'your',
  'toenails',
  'active',
  'emollients',
  'penetrate',
  'below',
  'surface',
  'layer',
  'skin',
  'provide',
  'deephydration',
  'increase',
  'moisture',
  'levels',
  'your',
  'toenails',
  'stopping',
  'them',
  'from',
  'becoming',
  'dry',
  'thick',
  'hard'],
 ['quadratini',
  'dark',
  'chocolate',
  'bite',
  'size',
  'wafer',
  'cookies',
  'snacks',
  'sweets',
  'cookies',
  'biscuits',
  'subtly',
  'bitter',
  'dark',
  'pleasure',
  'cu

In [ ]:
dictionary = corpora.Dictionary(docs)
dictionary.save(os.path.join('output_data', 'products_corpus.dict'))

In [222]:
print(dictionary)

Dictionary(119 unique tokens: ['a08', 'add', 'available', 'beef', 'braised']...)


In [223]:
print(dictionary.token2id)

{'a08': 0, 'add': 1, 'available': 2, 'beef': 3, 'braised': 4, 'chicken': 5, 'combo': 6, 'cuisine': 7, 'cuttlefish': 8, 'department': 9, 'fried': 10, 'golden': 11, 'mchefz': 12, 'mchefzcuisine': 13, 'menu': 14, 'nasi': 15, 'ons': 16, 'rice': 17, 'served': 18, 'super': 19, 'thigh': 20, 'with': 21, '20180730brbr': 22, 'at': 23, 'away': 24, 'bars': 25, 'breakfast': 26, 'cereal': 27, 'cereals': 28, 'close': 29, 'consumption': 30, 'cupboard': 31, 'date': 32, 'date—dont': 33, 'enjoy': 34, 'expiry': 35, 'food': 36, 'from': 37, 'get': 38, 'getting': 39, 'honey': 40, 'is': 41, 'it': 42, 'its': 43, 'let': 44, 'lower': 45, 'now': 46, 'pricebrbrexpiry': 47, 'product': 48, 'safe': 49, 'stars': 50, 'still': 51, 'this': 52, 'you': 53, 'combination': 54, 'cooling': 55, 'drinks': 56, 'finish': 57, 'flavour': 58, 'flavours': 59, 'fruity': 60, 'ice': 61, 'leaves': 62, 'lemon': 63, 'real': 64, 'refreshing': 65, 'stimulating': 66, 'sweet': 67, 'taste': 68, 'tea': 69, '20181013': 70, 'anticavity': 71, 'beaut

In [224]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
corpora.MmCorpus.serialize(os.path.join('output_data', 'products_corpus.mm'), corpus)

In [225]:
!ls 'output_data'

deerwester.dict          products_corpus.mm
products_corpus.dict     products_corpus.mm.index


In [226]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [227]:
! ls 'output_data'

deerwester.dict          products_corpus.mm
products_corpus.dict     products_corpus.mm.index


In [228]:
DATA_FOLDER = 'output_data'
dictionary = corpora.Dictionary.load(os.path.join(DATA_FOLDER, 'products_corpus.dict'))
corpus = corpora.MmCorpus(os.path.join(DATA_FOLDER, 'products_corpus.mm'))

In [229]:
print(corpus)

MmCorpus(854 documents, 119 features, 6134 non-zero entries)


In [247]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

In [248]:
lst  = ['wafer',
  'crunch',
  'chocolate',
  'hazelnuts',
  'members',
  'specials',
  'snacks',
  'sweets',
  'this',
  'satisfyingly',
  'crunchy',
  'wafer',
  'from',
  'ego',
  'has',
  'rich',
  'chocolate',
  'hazelnut',
  'flavoured',
  'cream',
  'filling',
  'that',
  'adds',
  'luxurious',
  'edge',
  'thin',
  'dry',
  'wafer',
  'biscuits',
  'these',
  'wafer',
  'biscuits',
  'make',
  'perfect',
  'accompaniment',
  'afternoon',
  'or',
  'morning',
  'tea',
  'or',
  'you',
  'can',
  'eat',
  'them',
  'whenever',
  'you',
  'need',
  'quick',
  'fruity',
  'sugar',
  'hit',
  'chocolate',
  'hazelnut',
  'cream',
  'filling',
  'makes',
  'these',
  'wafer',
  'biscuits',
  'an',
  'excellent',
  'companion',
  'biscuit',
  'vanilla',
  'ice',
  'cream']
sample_doc = " ".join(x for x in lst)

In [249]:
vec_bow = dictionary.doc2bow(sample_doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[(0, 1.3856346901538346), (1, -0.17431319519117727)]


In [250]:
index = similarities.MatrixSimilarity(lsi[corpus])

/Users/vipul/anaconda3/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [251]:
sims = index[vec_lsi]

In [252]:
print(list(enumerate(sims)))

[(0, 0.62337655), (1, 0.7766958), (2, 0.7614512), (3, 0.7222885), (4, 0.974021), (5, 0.9908109), (6, 0.6786488), (7, 1.0), (8, 0.8413998), (9, 0.7676879), (10, 0.98323786), (11, 0.91988397), (12, 0.67917114), (13, 0.29274315), (14, 0.7392273), (15, 0.83568597), (16, 0.7903517), (17, 0.6199254), (18, 0.75300044), (19, 0.8629918), (20, 0.71276206), (21, 0.68156993), (22, 0.8465688), (23, 0.79941386), (24, 0.73979926), (25, 0.7404145), (26, 0.71580666), (27, 0.859067), (28, 0.71312577), (29, 0.9995579), (30, 0.9993812), (31, 0.9935784), (32, 0.996683), (33, 0.9873418), (34, 0.9065108), (35, 0.9953827), (36, 0.7682688), (37, 0.9744601), (38, 0.7321194), (39, 0.71258026), (40, 0.9287634), (41, 0.967098), (42, 0.6864328), (43, 0.6972004), (44, 0.8880698), (45, 0.9638693), (46, 0.6782429), (47, 0.8854658), (48, 0.6527262), (49, 0.821473), (50, 0.7931007), (51, 0.83253765), (52, 0.70974094), (53, 0.9759567), (54, 0.7976716), (55, 0.72686595), (56, 0.975914), (57, 0.9952136), (58, 0.8647453), (